In [1]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
from sklearn.metrics import roc_curve, auc

def check_read_inside_region(start, seq, region):
        read_end = start + len(seq)
        region_start = int(region.split(":")[1].split("-")[0])
        region_end = int(region.split(":")[1].split("-")[1])
        if start >= region_start and read_end <= region_end:
            return "in"
        else: 
            return "overlap"

warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/GSHD_HN01/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6

mode = "all"

path_to_main_output = os.path.join(outdir, PROJECT, output_version)
path_to_05_output = os.path.join(path_to_main_output, "PANCANCER05_output")
path_to_06_output = os.path.join(path_to_main_output, "PANCANCER06_output")
os.system(f"mkdir -p {path_to_06_output}")

all_files = [item for item in pathlib.Path(path_to_05_output).glob("*.candi_reads.csv")]
all_read_files = [item for item in pathlib.Path(path_to_05_output).glob("*.read_classification.csv")]

##### read countdf: raw count and in-read count number. 
countdf = pd.read_csv(os.path.join(path_to_05_output, "all_count.csv"), index_col = [0])

all_samples = []
all_mean_candi_reads = []
for file in all_files:
    tmpdf = pd.read_csv(file)
    mean_candi_reads = tmpdf.candi_reads.mean()
    all_samples.append(file.name.split(".")[0])
    all_mean_candi_reads.append(mean_candi_reads)

candidf = pd.DataFrame({"SampleID": all_samples, "num_candi_reads": all_mean_candi_reads})
candidf = candidf.merge(countdf, right_on = "SampleID", left_on = "SampleID")
candidf["ratio_raw"] = candidf["num_candi_reads"] / candidf["raw_count"]
candidf["ratio_in_reads"] = candidf["num_candi_reads"] / candidf["in_read_count"]
candidf.to_excel(os.path.join(path_to_06_output, "candi_reads_all_{}_vs_control.xlsx".format("PAN_CANCER")), index = False)


Countdf existst, reading in from /media/hieunguyen/GSHD_HN01/outdir/TMD450_TCGA_data_analysis/20240910/PANCANCER05_output/all_count.csv ...


In [2]:
candidf

,SampleID,num_candi_reads,raw_count,in_read_count,ratio_raw,ratio_in_reads
0,K4AA24,11.770349,18266,18266,0.000644,0.000644
1,MB170,1.884106,2846,2846,0.000662,0.000662
2,MH020,167.294798,178192,178192,0.000939,0.000939
3,MB070,6.276471,10629,10629,0.000591,0.000591
4,MB119,8.748521,12680,12680,0.000690,0.000690
...,...,...,...,...,...,...
1093,K0AAAF83,10.617647,17170,17170,0.000618,0.000618
1094,MH054,35.301170,39982,39982,0.000883,0.000883
1095,K0AAAF88R1,5.940828,9168,9168,0.000648,0.000648
1096,MB079,67.538905,112334,112334,0.000601,0.000601
